In [39]:
from itertools import combinations
from functools import reduce

### Day 1

In [12]:
with open('data/day01.txt', 'r+') as f:
    expense_report = [int(s) for s in f.read().split('\n') if s != '']

In [40]:
def expense_product(expense_report, n):
    ints = [c for c in combinations(expense_report, n) if sum(c) == 2020][0]
    return reduce((lambda x, y: x * y), ints)

In [42]:
expense_product(expense_report, 2) ## part 1

1005459

In [43]:
expense_product(expense_report, 3) ## part 2

92643264